# Optimising DataFrames and Using Chunk Processing

In [1]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = 99

## Part 1: Creating a function to determine appropriate size of batches of DF to use for processing, given the user's memory constraints.

In [2]:
def chunksize(file, mem_available, rows=100):
    
    '''User specifies the csv file with the data, the memory available in MB, 
    and optionally the chunksizes for each batch (default is set to 100 rows).
    User is returned the chunksize to use, the total number of rows of the DF,
    and the total memory of the DF.'''
    
    # Read in file with initial number of rows and the memory usage of a single
    # batch is computed.
    rows_chunk = pd.read_csv(file, nrows=rows)
    mem_usage = rows_chunk.memory_usage(deep=True).sum() / (1024 * 1024)
    
    # If initial number of rows wasn't too large, ideal row size is computed.
    # To stay on the conservative size, the ideal size is half of the available
    # memory.
    if mem_usage < (mem_available /2):
        while mem_usage < (mem_available / 2):
            rows += 100
            rows_chunk = pd.read_csv(file, nrows=rows)
            mem_usage = rows_chunk.memory_usage(deep=True).sum() / (1024 * 1024)
        
        # With the ideal chunksize determined, each batch is imported in loop to
        # calculate the total number of rows of the DF as well as the total memory
        # usage of the DF.
        chunk_iter = pd.read_csv(file, chunksize=(rows-100))
        chunk_rows = (rows - 100)
        total_rows = 0
        mem_usage_all = []
        for chunk in chunk_iter:
            total_rows += len(chunk)
            mem_usage_all.append(chunk.memory_usage(deep=True).sum() / 1024 ** 2)
        mem_usage_total = sum(mem_usage_all) 
        return ("Each chunk contains {} rows".format(rows-100), 
                "The DataFrame contains {} rows".format(total_rows),
               "The DataFrame uses {} MB of memory".format(mem_usage_total))
    
    # If initial number of rows was too large, a warning is returned to the user.
    else:
        return "Insufficient memory"

In [3]:
'''For this particular case, we'll use 10MB'''

# file you want to import
file = 'loans_2007.csv'
# memory available in MB
mem_available = 10

chunks = chunksize(file, mem_available)
chunks

('Each chunk contains 3200 rows',
 'The DataFrame contains 42538 rows',
 'The DataFrame uses 66.2467565536499 MB of memory')

In [4]:
'''Assigning the the chunksize and total number of rows in the DF to variables for future use'''

chunk_rows = [int(s) for s in chunks[0].split() if s.isdigit()][0]
total_rows = [int(s) for s in chunks[1].split() if s.isdigit()][0]

In [5]:
'''Inspect columns'''

rows_chunk = pd.read_csv(file, nrows=10).head()
rows_chunk

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,NaN,10+ years,RENT,24000.0,Verified,Dec-2011,Fully Paid,n,credit_card,Computer,860xx,AZ,27.65,0.0,Jan-1985,1.0,3.0,0.0,13648.0,83.7%,9.0,f,0.00,0.00,5863.155187,5833.84,5000.00,863.16,0.00,0.00,0.00,Jan-2015,171.62,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,1077430,1314167.0,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,30000.0,Source Verified,Dec-2011,Charged Off,n,car,bike,309xx,GA,1.00,0.0,Apr-1999,5.0,3.0,0.0,1687.0,9.4%,4.0,f,0.00,0.00,1008.710000,1008.71,456.46,435.17,0.00,117.08,1.11,Apr-2013,119.66,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,1077175,1313524.0,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,NaN,10+ years,RENT,12252.0,Not Verified,Dec-2011,Fully Paid,n,small_business,real estate business,606xx,IL,8.72,0.0,Nov-2001,2.0,2.0,0.0,2956.0,98.5%,10.0,f,0.00,0.00,3005.666844,3005.67,2400.00,605.67,0.00,0.00,0.00,Jun-2014,649.91,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,1076863,1277178.0,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,AIR RESOURCES BOARD,10+ years,RENT,49200.0,Source Verified,Dec-2011,Fully Paid,n,other,personel,917xx,CA,20.00,0.0,Feb-1996,1.0,10.0,0.0,5598.0,21%,37.0,f,0.00,0.00,12231.890000,12231.89,10000.00,2214.92,16.97,0.00,0.00,Jan-2015,357.48,Apr-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
4,1075358,1311748.0,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,University Medical Group,1 year,RENT,80000.0,Source Verified,Dec-2011,Current,n,other,Personal,972xx,OR,17.94,0.0,Jan-1996,0.0,15.0,0.0,27783.0,53.9%,38.0,f,461.73,461.73,3581.120000,3581.12,2538.27,1042.85,0.00,0.00,0.00,Jun-2016,67.79,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


## Part 2: Dropping columns for future analysis.

In [6]:
'''Looking at how many columns are numeric and how many are objects in each batch to check for inconsistencies'''

chunk_iter = pd.read_csv('loans_2007.csv', chunksize=chunk_rows)

numeric_cols = []
string_cols = []
for col in chunk_iter:
    numeric_cols.append(col.select_dtypes(include=[np.number]).shape[1])
    string_cols.append(col.select_dtypes(include=['object']).shape[1])
    
print("Numeric columns:\n {}".format(numeric_cols))
print("Object columns:\n {}".format(string_cols))

Numeric columns:
 [31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 30, 30]
Object columns:
 [21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 21, 22, 22]


### Clearly the last two columns are inconsistent with the rest of the data. It's worth taking a closer look.

In [7]:
'''Running a loop to see which columns are causing the inconsistencies.'''

chunk_iter = pd.read_csv('loans_2007.csv', chunksize=chunk_rows)

obj_cols = []
for i, chunk in enumerate(chunk_iter):
    # Only need to look at object columns.
    chunk_cols = chunk.select_dtypes(include=['object']).columns.tolist()
    if len(obj_cols) > 0:
        if len(chunk_cols) != len(obj_cols):
            print("Chunk {} out of {} has columns:\n {} \n".format(i+1, len(numeric_cols), chunk_cols), "\n")
    else:
        obj_cols = chunk_cols
        print("Columns for comparison:\n {}".format(obj_cols), "\n")

Columns for comparison:
 ['term', 'int_rate', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'purpose', 'title', 'zip_code', 'addr_state', 'earliest_cr_line', 'revol_util', 'initial_list_status', 'last_pymnt_d', 'last_credit_pull_d', 'application_type'] 

Chunk 13 out of 14 has columns:
 ['id', 'term', 'int_rate', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'purpose', 'title', 'zip_code', 'addr_state', 'earliest_cr_line', 'revol_util', 'initial_list_status', 'last_pymnt_d', 'last_credit_pull_d', 'application_type'] 
 

Chunk 14 out of 14 has columns:
 ['id', 'term', 'int_rate', 'grade', 'sub_grade', 'emp_title', 'emp_length', 'home_ownership', 'verification_status', 'issue_d', 'loan_status', 'pymnt_plan', 'purpose', 'title', 'zip_code', 'addr_state', 'earliest_cr_line', 'revol_util', 'initial_list_status', 'last_pymnt_d

### The 'id' column is numeric for all but the last two batches, where it then becomes an object column. As 'id' is not useful for data analysis or modelling, it will be dropped anyway. The (object) columns that will be kept for analysis or modelling are as follows:

In [8]:
obj_cols_to_keep = ['term', 'sub_grade', 'emp_title', 'home_ownership', 'verification_status', 'issue_d', 'purpose', 
 'earliest_cr_line', 'revol_util', 'last_pymnt_d', 'last_credit_pull_d']

## Part 3: Converting Relevant Object Columns to Integer or Float

From the earlier inspection, out of the remaining object columns the following can be converted to integer or float (downcast to smallest kind) to save space in memory.

- 'term'. The word "month" needs to be stripped.
- 'revol_util'. The "%" sign needs to be stripped.

First we check for null values to see if converting to integer dtypes is possible. Otherwise, we can use convert to float dtypes

In [9]:
chunk_iter = pd.read_csv('loans_2007.csv', chunksize=chunk_rows, usecols=['term', 'revol_util'])

obj_cols = []
for chunk in chunk_iter:
    obj_cols.append(chunk.apply(pd.isnull).sum())

obj_cols_df = pd.concat(obj_cols).groupby(pd.concat(obj_cols).index).sum()
obj_cols_df

revol_util    93
term           3
dtype: int64

Looks like, we'll have to convert these two columns to a float dtype.

In [10]:
# Note since only object columns are needed, we'll only import the useful ones in the list above.
chunk_iter = pd.read_csv('loans_2007.csv', chunksize=chunk_rows, usecols=obj_cols_to_keep)

for chunk in chunk_iter:
    
    term_num = pd.to_numeric(chunk['term'].str.rstrip("months"), downcast='float')
    chunk['term_cleaned'] = term_num
    
    revol_util_num = pd.to_numeric(chunk['revol_util'].str.rstrip("%"), downcast='float')
    chunk['revol_util_cleaned'] = revol_util_num

## Part 4: Converting Relevant Object Columns to Dtype Category

First we need to see which category columns have a small number of unique values, so the category dtype conversion will be useful.

In [11]:
'''Obtaining a dictionary of object columns to convert to dtype category to further save on space. 
This dictionary can be used in Pandas' read_csv method'''

chunk_iter = pd.read_csv('loans_2007.csv', chunksize=chunk_rows)

unique_vals = dict()
for chunk in chunk_iter:
    chunk_objs = chunk.select_dtypes(include=['object'])
    cols_list = chunk_objs.columns.tolist()
    for col in cols_list:
        if col in unique_vals:
            unique_vals[col].append(chunk_objs[col].value_counts())
        else:
            unique_vals[col] = [chunk_objs[col].value_counts()]

unique_vals_grouped = dict()
str_col_unique = {}

for key in unique_vals:
    concat_vals = pd.concat(unique_vals[key])
    unique_vals_grouped[key] = concat_vals.groupby(concat_vals.index).sum()
    if unique_vals_grouped[key].shape[0] / total_rows < 0.5:
        str_col_unique[key] = unique_vals_grouped[key].shape[0]

str_col_unique

{'term': 2,
 'int_rate': 394,
 'grade': 7,
 'sub_grade': 35,
 'emp_length': 11,
 'home_ownership': 5,
 'verification_status': 3,
 'issue_d': 55,
 'loan_status': 9,
 'pymnt_plan': 2,
 'purpose': 14,
 'title': 21264,
 'zip_code': 837,
 'addr_state': 50,
 'earliest_cr_line': 530,
 'revol_util': 1119,
 'initial_list_status': 1,
 'last_pymnt_d': 103,
 'last_credit_pull_d': 108,
 'application_type': 1,
 'id': 4138}

### The following columns will therefore be changed to category. Note 'term' is excluded because it's been decided to convert that column to dtype float.

In [12]:
cat_cols = {
    'sub_grade': 'category', 
    'home_ownership': 'category',
    'verification_status': 'category',
    'purpose': 'category'
}

## Part 5: Converting Relevant Object Columns to Datetime

This is easy enough from the initial inspection.

In [13]:
'''Columns that can be converted from object to datetime to further save on memory'''

date_cols = ['issue_d', 'earliest_cr_line', 'last_pymnt_d', 'last_credit_pull_d']

## Part 6: Converting Relevant Float Columns to Integer Dtypes and Downcasting Float Columns

First we need to check for null values, since only columns without any null values can be converted to an integer dtype (assuming the values themselves can be represented by integers).

In [14]:
chunk_iter = pd.read_csv('loans_2007.csv', chunksize=chunk_rows)

float_cols = []
for chunk in chunk_iter:
    float_df = chunk.select_dtypes(include=['float'])
    float_cols.append(float_df.apply(pd.isnull).sum())

float_cols_df = pd.concat(float_cols).groupby(pd.concat(float_cols).index).sum()
float_cols_df

acc_now_delinq                  32
annual_inc                       7
chargeoff_within_12_mths       148
collection_recovery_fee          3
collections_12_mths_ex_med     148
delinq_2yrs                     32
delinq_amnt                     32
dti                              3
funded_amnt                      3
funded_amnt_inv                  3
inq_last_6mths                  32
installment                      3
last_pymnt_amnt                  3
loan_amnt                        3
member_id                        3
open_acc                        32
out_prncp                        3
out_prncp_inv                    3
policy_code                      3
pub_rec                         32
pub_rec_bankruptcies          1368
recoveries                       3
revol_bal                        3
tax_liens                      108
total_acc                       32
total_pymnt                      3
total_pymnt_inv                  3
total_rec_int                    3
total_rec_late_fee  

There are no float columns that can be converted to an integer dtype due to null values (remember, we're not using the 'id' column), so all float columns will just be downcast to the smallest float columns possible.

In [15]:
float_to_downcast = float_cols_df.index.tolist()
chunk_iter = pd.read_csv('loans_2007.csv', usecols=float_to_downcast, chunksize=chunk_rows)

for chunk in chunk_iter:
    for col in float_to_downcast:
        chunk[col] = pd.to_numeric(chunk[col], downcast='float')

## Part 7: Computing the New Memory Footprint to see Savings

Let's put all this code in the initial footprint and see how much memory we've saved. First, let's create a list of the columns we've decided to keep. We won't use this variable to our first comparison since we used all of the columns in the initial run of the function.

In [16]:
cols_to_keep = obj_cols_to_keep + float_to_downcast

In [17]:
def chunksize(file, mem_available, rows=100, cols_to_keep=None, date_cols=False, 
              cat_cols=None):
    
    '''User specifies the csv file with the data, the memory available in MB, 
    and optionally the chunksizes for each batch (default is set to 100 rows).
    Now optional inputs also include the columns in the DF to be used (if known),
    the columns to convert to datetime dtype, 
    User is returned the chunksize to use, the total number of rows of the DF,
    and the total memory of the DF.'''
    
    # Read in file with initial number of rows and the memory usage of a single
    # batch is computed.
    rows_chunk = pd.read_csv(file, nrows=rows)
    mem_usage = rows_chunk.memory_usage(deep=True).sum() / (1024 * 1024)
    
    # If initial number of rows wasn't too large, ideal row size is computed.
    # To stay on the conservative size, the ideal size is half of the available
    # memory.
    if mem_usage < (mem_available /2):
        while mem_usage < (mem_available / 2):
            rows += 100
            rows_chunk = pd.read_csv(file, nrows=rows)
            mem_usage = rows_chunk.memory_usage(deep=True).sum() / (1024 * 1024)
        
        # With the ideal chunksize determined, each batch is imported in loop to
        # calculate the total number of rows of the DF as well as the total memory
        # usage of the DF.
        
        float_cols_df.index.tolist()
        
        chunk_iter = pd.read_csv(file, chunksize=(rows-100), usecols=cols_to_keep, parse_dates=date_cols,
                                dtype=cat_cols)
        chunk_rows = (rows - 100)
        total_rows = 0
        mem_usage_all = []

        for chunk in chunk_iter:
            total_rows += len(chunk)
            
            # Change relevant object columns to dtype float
            term_num = pd.to_numeric(chunk['term'].str.rstrip("months"), downcast='float')
            chunk['term_cleaned'] = term_num
            revol_util_num = pd.to_numeric(chunk['revol_util'].str.rstrip("%"), downcast='float')
            chunk['revol_util_cleaned'] = revol_util_num
            
            # Downcast float columns
            float_cols = chunk.select_dtypes(include=['float'])
            for col in float_cols.columns.tolist():
                chunk[col] = pd.to_numeric(chunk[col], downcast='float')
            
            mem_usage_all.append(chunk.memory_usage(deep=True).sum() / 1024 ** 2)
        mem_usage_total = sum(mem_usage_all) 
        return ("Each chunk contains {} rows".format(rows-100), 
                "The DataFrame contains {} rows".format(total_rows),
               "The DataFrame uses {} MB of memory".format(mem_usage_total))
    
    # If initial number of rows was too large, a warning is returned to the user.
    else:
        return "Insufficient memory"

In [18]:
chunksize('loans_2007.csv', 10, rows=100, date_cols=date_cols, cat_cols=cat_cols)

('Each chunk contains 3200 rows',
 'The DataFrame contains 42538 rows',
 'The DataFrame uses 42.1108922958374 MB of memory')

Success! We've saved about 24MB of memory (the initial DF was about 66MB) if we keep using all the columns! Of course, if we use only the ones we think will be useful for analysis and modelling we'll save a lot more. Let's check how much.

In [19]:
chunksize('loans_2007.csv', 10, rows=100, cols_to_keep=cols_to_keep, date_cols=date_cols, cat_cols=cat_cols)

('Each chunk contains 3200 rows',
 'The DataFrame contains 42538 rows',
 'The DataFrame uses 14.89106273651123 MB of memory')

So we've gone from over 66MB of memory to under 15MB. A reduction of around 77% of memory footprint!